In [1]:
import sys,os
from datetime import datetime
import pandas as pd
import numpy as np
from pybedtools import BedTool, example_bedtool
from math import pow, exp

In [ ]:
##########################################
# generate CTCF pair from CTCF peak list #
##########################################
def peakToPair(df_peak, f_out):
    print(datetime.now())
    #df_peak = pd.read_csv(f_in,sep='\t',header=None)
    ncol = len(df_peak.columns)
    df_peak.columns = range(len(df_peak.columns))
    df_peak['tmp_index'] = df_peak.index
    dic_peak = df_peak.set_index('tmp_index').T.to_dict()

    pbt_peak = BedTool.from_dataframe(df_peak).slop(b=500000,genome="hg38")
    df_peak_intersect = pbt_peak.intersect(pbt_peak,wo=True).to_dataframe(disable_auto_names=True, header=None)
    df_peak_intersect.columns = range(len(df_peak_intersect.columns))
    df_peak_intersect = df_peak_intersect[df_peak_intersect[ncol]<df_peak_intersect[ncol*2+1]].reset_index()

    df_peak_list1 = pd.DataFrame(df_peak_intersect[ncol].map(dic_peak).tolist())
    df_peak_list2 = pd.DataFrame(df_peak_intersect[ncol*2+1].map(dic_peak).tolist())
    df_peak_list1.columns = range(len(df_peak_list1.columns))
    df_peak_list2.columns = range(len(df_peak_list2.columns))

    df_peak_pair = pd.concat([df_peak_list1.iloc[:,0:3],df_peak_list2.iloc[:,0:3],df_peak_list1.iloc[:,3:ncol],df_peak_list2.iloc[:,3:ncol]],axis=1)
    print(datetime.now())
    if(f_out==0):
        return df_peak_pair
    else:
        df_peak_pair.to_csv(f_out,sep='\t',index=False,header=False)
    

In [ ]:
peakToPair('CTCF_cohesin_consensus_20k_300_hg38.bed','CTCF_300_pair_1Mb_hg38.bedpe')
peakToPair('CTCF_cohesin_consensus_20k_500_hg38.bed','CTCF_500_pair_1Mb_hg38.bedpe')
peakToPair('CTCF_cohesin_consensus_20k_1k_hg38.bed','CTCF_1k_pair_1Mb_hg38.bedpe')

In [ ]:
###################################
# identify CTCF motif orientation #
###################################

peakname = 'CTCF_cohesin_consensus_20k_300_hg38'

#os.system('python2 ~/work/scripts/fetchseqs.py '+peakname+'.bed '+peakname+'.fa -d ~/work/genomes/hg38')
#os.system('storm -m -s '+peakname+'.fa ~/Desktop/software/CTCF.mat > '+peakname+'.CTCFmotif.txt')

df_in = pd.read_csv(peakname+'.CTCFmotif.txt',sep=' ',header=None,names=range(9),usecols=[0,3,7,8])
df_motif = df_in[df_in[0]=='BS'][[3,7,8]].replace(';', '',regex=True).drop_duplicates()
df_motif.columns = ['index','strand','score']
df_motif['score'] = df_motif['score'].astype(int)
#df_motif['score'] = df_motif['score'].round(3)
df_peak = pd.read_csv(peakname+'.bed',sep='\t',header=None)
df_peak['index'] = df_peak[0]+':'+(df_peak[1]+1).astype(str)+'-'+df_peak[2].astype(str)

dic_motif = df_motif.set_index('index').T.to_dict()
df_peak[['score','strand']] = pd.DataFrame(df_peak['index'].map(dic_motif).tolist())

df_peak['index'] = df_peak.index
df_peak.to_csv(peakname+'.CTCFmotif.bed',sep='\t',index=False,header=False)
'''
df_peak.columns = ['chr','s','e','index','score','strand']
df_peak['strand'] = df_peak['strand'].replace(to_replace=['p', 'n'], value=[1, -1])

df_in = df_peak[:]
df_peak_pair = peakToPair(df_in,0)
df_peak_pair.columns = ['chr1','s1','e1','chr2','s2','e2','index1','score1','strand1','index2','score2','strand2']
df_peak_pair[['chr1','s1','e1','chr2','s2','e2','score1','strand1','score2','strand2']].to_csv('CTCF_300_pair_1Mb_hg38.CTCFmotif.bedpe',sep='\t',index=False,header=False)
'''

In [ ]:
def CTCF_binding_intensity(bw_file,bed_file,cell):
    peak_size = '300'
    df_peak = pd.read_csv('CTCF_cohesin_consensus_20k_'+peak_size+'_hg38.bed',sep='\t',header=None)
    df_macs = pd.read_csv('/data/mbeer3/data/'+bed_file,sep='\t',header=None)

    df_summit = pd.DataFrame({'chr': df_macs[0], \
                          's': df_macs[1]+df_macs[9], \
                          'e': df_macs[1]+df_macs[9]+1, \
                          'strand': '.', \
                          'score': df_macs[6]}).sort_values(['chr','s'])

    pbt_peak = BedTool.from_dataframe(df_peak)
    pbt_summit = BedTool.from_dataframe(df_summit)
    df_peak_summit = pbt_peak.map(pbt_summit,o='max').to_dataframe(disable_auto_names=True, header=None).replace('.',0)
    df_peak_summit[3].to_csv('ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_MACSscore.txt',sep='\t',index=False,header=False)
    
    os.system('bigWigAverageOverBed /data/mbeer3/data/'+bw_file+' CTCF_cohesin_consensus_20k_'+peak_size+'_hg38.CTCFmotif.bed '+'ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_readcount.txt')


In [ ]:
def LEmodel(peak_size, bw_file, cell, kd_scale, w, CTCF_BI):

    #os.system('bigWigAverageOverBed /data/mbeer3/data/'+bw_file+' CTCF_cohesin_consensus_20k_'+peak_size+'_hg38.CTCFmotif.bed '+'ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_readcount.txt')
    if(CTCF_BI == 'readcount'):
        df_ctcf_chip = pd.read_csv('ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_readcount.txt',sep='\t',header=None,usecols=[0,3])
        df_ctcf_chip.columns = ['id','count']
        df_ctcf_chip = df_ctcf_chip.sort_values('id').reset_index()
    else:
        df_ctcf_chip = pd.read_csv('ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_MACSscore.txt',sep='\t',header=None)
        df_ctcf_chip.columns = ['count']
        
    kd = df_ctcf_chip['count'].mean()*kd_scale

    df_peak = pd.read_csv('CTCF_cohesin_consensus_20k_'+peak_size+'_hg38.CTCFmotif.bed',sep='\t',header=None)
    df_peak.columns = ['chr','s','e','index','score','strand']
    df_peak['strand'] = df_peak['strand'].replace(to_replace=['p', 'n'], value=[1, -1])
    df_peak['index'] = df_peak.index+1
    df_peak['score'][df_peak['score'] < 0] = 0
    df_peak['p_i'] = df_ctcf_chip['count']/(df_ctcf_chip['count']+kd)
    
    df_in = df_peak[:]
    df_peak_pair = peakToPair(df_in,0)
    df_peak_pair.columns = ['chr1','s1','e1','chr2','s2','e2','index1','score1','strand1','p_i','index2','score2','strand2','p_j']
    df_peak_pair['p_ij'] = df_peak_pair['p_i']*df_peak_pair['p_j']
    df_peak_pair['ori'] = np.power(w,(df_peak_pair['strand1']-df_peak_pair['strand2']-2)/2)

    print(datetime.now())
    df_peak_pair['lc'] = df_peak_pair.apply(lambda x: np.prod([1-df_peak.loc[i,'p_i'] for i in range(x['index1'],x['index2']-1)]), axis=1)
    df_peak_pair['le_prediction'] = df_peak_pair['p_ij']*df_peak_pair['ori']*df_peak_pair['lc']
    print(datetime.now())
    
    df_peak_pair[['p_ij','ori','lc','le_prediction']].to_csv('ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_LEmodel.'+CTCF_BI+'.txt',index=False,header=False,sep='\t')
    #os.system('rm ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_readcount.txt')

In [ ]:
#LEmodel('300', 'ENCFF541CSJ.bigWig', 'CTCF_1')

In [ ]:
df_cell = pd.read_csv('ENCODE_CTCF_list.txt',sep='\t',header=None)
sample_name = open('sample_name.txt').readline().split()
cell_list = pd.DataFrame(pd.Series(sample_name).str.split('_',1, expand=True))[0].drop_duplicates().tolist()

In [ ]:
############################
#run LE model for all cell line#
############################

for i,x in df_cell.iterrows():
    if(x[5] not in cell_list):
        continue
    #if(not os.path.isfile('ENCODE_'+x[0]+'_300_pair_1Mb_hg38_readcount.txt')):
        #f_out.write('CTCF_binding_intensity(\''+x[3]+'\',\''+x[1]+'\',\''+x[0]+'\')\n')
        #CTCF_binding_intensity(x[3],x[1],x[0])
    f_out = open('qf1','w')
    f_out.write('#!/bin/bash\n')
    f_out.write('#SBATCH --time=48:0:0\n')
    f_out.write('#SBATCH --mem=12G\n\n')

    f_out.write('python LEmodel_predict.py 300 '+x[3]+' '+x[0]+' 6.0 3.0 MACSscore'+'\n')
    f_out.close()
    os.system('sbatch qf1')


In [ ]:
############################################
#run LE model grid search for all cell line#
############################################
x = df_cell[17:18].values.tolist()[0]
for kd in range(2,22,2):
    for w in range(7,25,2):
        f_out = open('qf1','w')
        f_out.write('#!/bin/bash\n')
        f_out.write('#SBATCH --time=48:0:0\n')
        f_out.write('#SBATCH --mem=12G\n\n')
        #print('python LEmodel_predict.py 300 '+x[3]+' '+x[0]+' '+str(kd/2)+' '+str(w/5)+' '+'readcount'+'\n')
        f_out.write('python LEmodel_predict.py 300 '+x[3]+' '+x[0]+' '+str(kd/2)+' '+str(w/5)+' '+'readcount'+'\n')
        f_out.write('python LEmodel_predict.py 300 '+x[3]+' '+x[0]+' '+str(kd/2)+' '+str(w/5)+' '+'MACSscore'+'\n')
        f_out.close()
        os.system('sbatch qf1')

In [ ]:
################################
#merge read count of all sample#
################################

df_out = pd.DataFrame()
peak_size = '300'
for i,x in df_cell.iterrows():
    cell = x[0]
    df_in = pd.read_csv('ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_readcount.txt',sep='\t',header=None)
    df_out[x[0]] = df_in[3].round(1)

df_out.to_csv('all_ENCODE_CTCF_'+peak_size+'_pair_1Mb_hg38_readcount.txt',index=False,sep='\t')

In [ ]:
#########################################
#merge LE model prediction of all sample#
#########################################

df_out = pd.DataFrame()
peak_size = '300'
for i,x in df_cell.iterrows():
    cell = x[0]
    #print(x[5])
    #if(x[5] not in cell_list):
    #    os.system('rm ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_LEmodel.txt')
    
    if(x[5] in cell_list):
        df_in = pd.read_csv('ENCODE_'+cell+'_'+peak_size+'_pair_1Mb_hg38_LEmodel_6.0_3.0_readcount.txt',sep='\t',header=None)
        #print('LE_'+x[5]+'_'+x[0].split('_')[1])
        df_out['LE_'+x[5]+'_'+x[0].split('_')[1]] = (df_in[0]*df_in[2]*100).round(6)
    
#df_out.to_csv('all_ENCODE_CTCF_'+peak_size+'_pair_1Mb_hg38_LEmodel.txt',index=False,sep='\t')
#df_out.to_csv('all_ChIAPET_CTCF_'+peak_size+'_pair_1Mb_hg38_LEmodel.txt',index=False,sep='\t')

In [ ]:
df_out.to_csv('all_ChIAPET_CTCF_'+peak_size+'_pair_1Mb_hg38_LEmodel_6.0_3.0_readcount.txt',index=False,sep='\t')

In [2]:
#########################################
# sort bam, bam2bedpe, remove duplicate #
#########################################

def removeDuplicate(prefix_readpair):

    print(datetime.now())
    #prefix_readpair = 'K562_CTCF_1'
    os.system('samtools sort -n -o '+prefix_readpair+'_hg38_sorted.bam --threads 4 '+prefix_readpair+'_hg38.bam')
    print('Finish sorting bam')
    os.system('bedtools bamtobed -i '+prefix_readpair+'_hg38_sorted.bam -bedpe > '+prefix_readpair+'_hg38_sorted.bedpe')
    print('Finish bam to bedpe')
    
    df = pd.read_csv(prefix_readpair+'_hg38_sorted.bedpe',sep='\t',header=None,usecols=[0,1,2,3,4,5])
    df = df[df[0]==df[3]]
    df_sort = df.sort_values([0,1,3,4])
    print('Finish sorting bedpe')
    
    df_uniq = df_sort.drop_duplicates()
    df_uniq.to_csv(prefix_readpair+'_hg38_sorted.rmdup.intra.bedpe',sep='\t',index=False,header=False)
    print('Finish removing duplicate')
    
    os.system('rm '+prefix_readpair+'_hg38_sorted.bam')
    os.system('rm '+prefix_readpair+'_hg38_sorted.bedpe')
    print(datetime.now())

In [ ]:
removeDuplicate('K562_CTCF_1')

In [23]:
prefix_list = list(os.popen('ls *_CTCF_r*bam|sed \'s/hg38.bam//g\''))
prefix_list

[]

In [16]:
for i in range(len(prefix_list)):
    prefix = prefix_list[i].strip('\n')
    ofile = open('qsub_'+prefix,'w')
    ofile.write('#!/bin/bash\n')
    ofile.write('#SBATCH --time=48:0:0\n')
    ofile.write('#SBATCH --mem=36G\n')
    ofile.write('python3 process_bam.py '+prefix+'\n')
    ofile.write('')
    ofile.close()
    #os.system("sbatch qLASSOnf")

1
2
3
4
5
6
7
8
9
10


In [ ]:
#############################
# map PET read to loop list #
#############################

def countPET(prefix_peakpair,prefix_readpair):
    #prefix_peakpair = 'CTCF_300_pair_1Mb_hg38'
    #prefix_readpair = 'K562_CTCF_1'
    print(datetime.now())
    print(prefix_peakpair,prefix_readpair)
    
    df_ctcfpair = pd.read_csv(prefix_peakpair+'.bedpe',sep='\t',header=None)
    df_ctcfpair['index'] = df_ctcfpair.index
    df_readpair = pd.read_csv(prefix_readpair+'_hg38_sorted.rmdup.intra.bedpe',sep='\t',header=None)
    print('Finish reading bedpe')
    
    pbt_uniq = BedTool.from_dataframe(df_readpair)
    pbt_ctcfpair = BedTool.from_dataframe(df_ctcfpair)
    pbt_intersect = pbt_ctcfpair.pair_to_pair(pbt_uniq)
    print('Finish intersecting bedpe')
    
    df_intersect = pbt_intersect.to_dataframe(disable_auto_names=True, header=None)
    df_ctcfpair['PET'] = df_ctcfpair['index'].map(df_intersect[6].value_counts()).fillna(0).astype(int)
    df_ctcfpair['PET'].to_csv(prefix_peakpair+'_PETcount_'+prefix_readpair+'.txt',sep='\t',index=False,header=False)
    print(datetime.now())

In [ ]:
countPET('CTCF_300_pair_1Mb_hg38','K562_CTCF_1')